# Power Grid Modelling

In [1]:
import os
import arrow
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import pandapower as pp
import pandapower as pp

from pandapower import networks
from pandapower import plotting as pp_plot

plt.style.use('tableau-colorblind10')

In [2]:
from GridExecutor import Executor

In [3]:
cwd = os.getcwd()
print(cwd)
data_fn = os.path.join(cwd, 'data_modelling')
grid_fn = os.path.join(cwd, 'grid_modelling')
fig_fn = os.path.join(cwd, 'figs')
print(data_fn)
print(grid_fn)

/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator
/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator/data_modelling
/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator/grid_modelling


In [4]:
fn = os.path.join(data_fn, 'main_agg.h5')
all_data = pd.read_hdf(fn, key='df')

In [5]:
start_date = '2020-01-01 00:00:00'
end_date = '2022-12-31 23:45:00'

In [6]:
data = all_data.loc[start_date:end_date].copy()

In [7]:
timestamps = [x[0] for x in data.index]

In [8]:
timestamps = list(dict.fromkeys(timestamps))

In [9]:
data.tail()

name    p_mw    q_mvar
2022-12-31 23:45:00 9   AGG_BUS_10  0.0000  0.000000
                    10  AGG_BUS_11  0.1882  0.091149
                    11  AGG_BUS_12  0.0000  0.000000
                    12  AGG_BUS_13  0.0808  0.039133
                    13  AGG_BUS_14  0.0404  0.019567

## Actual Grid Mdf_dataing

In [10]:
print('PandaPower Version: {}'.format(pp.__version__))

PandaPower Version: 2.2.1


In [11]:
grid = pp.networks.create_cigre_network_mv(with_der=False)

In [12]:
# drop the predefined load
grid.load = grid.load[0:0]

In [13]:
# run power flow
pp.runpp(grid)

## Define Load Mapping 

In [14]:
for bus_id in range(1,15):
    pp.create_load(grid, bus=bus_id, p_mw=0., q_mvar=0., name='AGG_BUS_%d' % bus_id, scaling=1., in_service=True, controllable=False)

In [15]:
grid.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,controllable
0,AGG_BUS_1,1,0.0,0.0,0.0,0.0,NaN,1.0,True,None,False
1,AGG_BUS_2,2,0.0,0.0,0.0,0.0,NaN,1.0,True,None,False
2,AGG_BUS_3,3,0.0,0.0,0.0,0.0,NaN,1.0,True,None,False
3,AGG_BUS_4,4,0.0,0.0,0.0,0.0,NaN,1.0,True,None,False
4,AGG_BUS_5,5,0.0,0.0,0.0,0.0,NaN,1.0,True,None,False
5,AGG_BUS_6,6,0.0,0.0,0.0,0.0,NaN,1.0,True,None,False
6,AGG_BUS_7,7,0.0,0.0,0.0,0.0,NaN,1.0,True,None,False
7,AGG_BUS_8,8,0.0,0.0,0.0,0.0,NaN,1.0,True,None,False
8,AGG_BUS_9,9,0.0,0.0,0.0,0.0,NaN,1.0,True,None,False
9,AGG_BUS_10,10,0.0,0.0,0.0,0.0,NaN,1.0,True,None,False


In [16]:
duration = (366 + 365 + 365) * 96
print('Duration: {} 1/4 hours'.format(duration))

Duration: 105216 1/4 hours


In [17]:
assert len(timestamps) >= duration 

In [18]:
executor = Executor(grid)

### Simulate 2020, 2021 and 2022

In [19]:
for t in range(duration):
    # run simulation
    try:
        ts = timestamps[t]
        frame = data.loc[ts]
        grid = executor.step(frame)
                
        if t%10000 == 0:
            print('Curent step: {}'.format(t))

    except Exception as e:
        print('Exception!')
        print(e)

Curent step: 0
Curent step: 10000
Curent step: 20000
Curent step: 30000
Curent step: 40000
Curent step: 50000
Curent step: 60000
Curent step: 70000
Curent step: 80000
Curent step: 90000
Curent step: 100000


In [20]:
simulation_data = executor.get_simulation_data()

In [21]:
assert len(simulation_data) == len(timestamps)

In [22]:
simulation_data.head()

,line_1_1_loading,line_2_3_loading,line_3_4_loading,line_4_5_loading,line_5_6_loading,line_7_8_loading,line_8_9_loading,line_9_10_loading,line_10_11_loading,line_3_8_loading,...,bus_13_q_mvar,bus_14_q_mvar,switch_0_status,switch_1_status,switch_2_status,switch_3_status,switch_4_status,switch_5_status,switch_6_status,switch_7_status
0,24.644473,20.131101,8.404872,7.058014,2.865729,4.337964,7.136024,2.829874,2.875793,12.656436,...,-0.026153,-0.013077,True,False,False,True,False,True,True,True
1,21.790400,17.948475,7.670718,6.389197,2.648196,4.053554,6.380672,2.528290,2.573173,11.468529,...,-0.034677,-0.017339,True,False,False,True,False,True,True,True
2,19.278630,16.028356,7.015811,5.801115,2.449640,3.786643,5.728396,2.268028,2.311765,10.422091,...,-0.041845,-0.020923,True,False,False,True,False,True,True,True
3,17.169135,14.410755,6.460969,5.303823,2.280746,3.558796,5.178226,2.048673,2.091190,9.537306,...,-0.047754,-0.023877,True,False,False,True,False,True,True,True
4,15.598922,13.203294,6.040075,4.931252,2.150115,3.378654,4.772639,1.887093,1.928513,8.873382,...,-0.051919,-0.025960,True,False,False,True,False,True,True,True


In [23]:
simulation_data.index = timestamps

In [24]:
simulation_data.head()

,line_1_1_loading,line_2_3_loading,line_3_4_loading,line_4_5_loading,line_5_6_loading,line_7_8_loading,line_8_9_loading,line_9_10_loading,line_10_11_loading,line_3_8_loading,...,bus_13_q_mvar,bus_14_q_mvar,switch_0_status,switch_1_status,switch_2_status,switch_3_status,switch_4_status,switch_5_status,switch_6_status,switch_7_status
2020-01-01 00:00:00,24.644473,20.131101,8.404872,7.058014,2.865729,4.337964,7.136024,2.829874,2.875793,12.656436,...,-0.026153,-0.013077,True,False,False,True,False,True,True,True
2020-01-01 00:15:00,21.790400,17.948475,7.670718,6.389197,2.648196,4.053554,6.380672,2.528290,2.573173,11.468529,...,-0.034677,-0.017339,True,False,False,True,False,True,True,True
2020-01-01 00:30:00,19.278630,16.028356,7.015811,5.801115,2.449640,3.786643,5.728396,2.268028,2.311765,10.422091,...,-0.041845,-0.020923,True,False,False,True,False,True,True,True
2020-01-01 00:45:00,17.169135,14.410755,6.460969,5.303823,2.280746,3.558796,5.178226,2.048673,2.091190,9.537306,...,-0.047754,-0.023877,True,False,False,True,False,True,True,True
2020-01-01 01:00:00,15.598922,13.203294,6.040075,4.931252,2.150115,3.378654,4.772639,1.887093,1.928513,8.873382,...,-0.051919,-0.025960,True,False,False,True,False,True,True,True


### Reduce simulation data

In [8]:
cols = [s for s in simulation_data.columns if "loading" in s]
print(cols)

['line_1_1_loading', 'line_2_3_loading', 'line_3_4_loading', 'line_4_5_loading', 'line_5_6_loading', 'line_7_8_loading', 'line_8_9_loading', 'line_9_10_loading', 'line_10_11_loading', 'line_3_8_loading', 'line_12_13_loading', 'line_13_14_loading', 'line_6_7_loading', 'line_11_14_loading', 'line_14_8_loading', 'trafo_0_loading', 'trafo_1_loading']


In [9]:
simulation_data_reduced = simulation_data[cols].copy()

## Export Data

In [11]:
h5_fn_r = os.path.join(data_fn, 'simulation_data_reduced.h5')
print(h5_fn_r)
simulation_data_reduced.to_hdf(h5_fn_r, mode='w', key='df')

/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator/data_modelling/simulation_data_reduced.h5


In [25]:
h5_fn = os.path.join(data_fn, 'simulation_data.h5')
print(h5_fn)
simulation_data.to_hdf(h5_fn, mode='w', key='df')

/Users/torge/Development/master/masterthesis_code/02_Experimente/DataGenerator/data_modelling/simulation_data.h5


## Save CIGRE Net

In [26]:
# Save net as JSON
filename = os.path.join(grid_fn, 'CIGRE_net.json')
pp.to_json(grid,filename)